In [1]:
import time

import torch
import torch.nn.utils.prune as prune
import torchvision.models as models
from torch.onnx import export

In [2]:
def model_to_onnx(
    model,
    output_file,
    input_shape=(1, 3, 224, 224),
):
    model.eval()
    input_tensor = torch.randn(input_shape)
    input_names = ["input"]
    output_names = ["output"]
    export(model, input_tensor, output_file, verbose=False, input_names=input_names, output_names=output_names)
    return output_file

In [3]:
# モデルを読み込む
model = models.resnet50(weights="IMAGENET1K_V2")
model_to_onnx(model, "resnet50_dense.onnx") # 密モデルを ONNX 形式で保存

input_image = torch.ones((1, 3, 224, 224))
output = model(input_image)  # ウォームアップ

start_time = time.time()
with torch.no_grad():
    output = model(input_image)
end_time = time.time()
print(f"推論時間（密）: {end_time - start_time:.4f} 秒")
# Intel Core i7-12700 で 0.0277 秒

推論時間（密）: 0.0274 秒


In [4]:
parameters_to_prune = [
    (module, "weight") for module in model.modules() if isinstance(module, torch.nn.Conv2d)
]  # すべての畳み込み層を枝刈り対象にする

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.9,
)  # 大域的・非構造・強度枝刈り

In [5]:
model(input_image)  # ウォームアップ

start_time = time.time()
with torch.no_grad():
    output = model(input_image)
end_time = time.time()
print(f"推論時間（枝刈り直後）: {end_time - start_time:.4f} 秒")
# Intel Core i7-12700 で 0.0359 秒
# マスクを都度適用しているのでかえって遅くなる

推論時間（枝刈り直後）: 0.0362 秒


In [6]:
# オプション: 再訓練をここで行う

In [7]:
for module in model.modules():
    if isinstance(module, torch.nn.Conv2d):
        prune.remove(module, "weight")  # 永続化

In [8]:
# ゼロ比率を表示
for module in model.modules():
    if isinstance(module, torch.nn.Conv2d):
        print(
            f"{module} Zero-Ratio: {100.0 * float(torch.sum(module.weight == 0)) / float(module.weight.nelement()):.2f}%"
        )

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False) Zero-Ratio: 40.20%
Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False) Zero-Ratio: 47.51%
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) Zero-Ratio: 68.17%
Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False) Zero-Ratio: 54.78%
Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False) Zero-Ratio: 51.41%
Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False) Zero-Ratio: 56.46%
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) Zero-Ratio: 63.88%
Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False) Zero-Ratio: 55.66%
Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False) Zero-Ratio: 53.83%
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) Zero-Ratio: 60.34%
Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False) Zero-Ratio: 60.27%
Conv2d(256, 128, kernel_size=(1, 1), str

In [9]:
output = model(input_image)  # ウォームアップ

start_time = time.time()
with torch.no_grad():
    output = model(input_image)
end_time = time.time()
print(f"推論時間（永続化後）: {end_time - start_time:.4f} 秒")
# Intel Core i7-12700 で 0.0270 秒
# マスクをパラメータに永続化したのでオーバーヘッドはなくなるが、疎計算に対応していないので、密計算と同じ速度

推論時間（永続化後）: 0.0291 秒


In [10]:
model_to_onnx(model, "resnet50_sparse.onnx") # 疎モデルを ONNX 形式で保存

'resnet50_sparse.onnx'